In [ ]:
#Import warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from collections import Counter
from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
# from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas
import numpy as np
import plotly.express as px

import datetime
import numpy as np
import plotly as pl
import matplotlib.pyplot as plt
# from imblearn.over_sampling import SMOTE
# from imblearn.under_sampling import ClusterCentroids
# from imblearn.combine import SMOTEENN
# from imblearn.metrics import classification_report_imbalanced

In [ ]:
# Load the data
file_path = Path('C:/Users/esobieski/Documents/Berkeley/TeamPySpark/loans.csv')
# df = pd.read_csv(file_path, skiprows=1)[:-2]
# df = df.loc[:, columns].copy()
loans_df = pd.read_csv(file_path)  # Can ddd in ( , index_col=0) if need to say which col becomes index
loans_df.head(10)

# Data Preprocessing

In [ ]:
# Set index
loans_df.set_index

In [ ]:
# Get all column names
loans_df.columns

In [ ]:
# Name variables to find time elapsed from when request was posted to when it was funded.
raised_time = pd.to_datetime(loans_df["RAISED_TIME"])
posted_time = pd.to_datetime(loans_df["POSTED_TIME"])
elapsed_time_df = raised_time - posted_time

In [ ]:
# Histogram of elapsed time amounts
elapsed_time_df.astype("timedelta64[D]").hist(range=[-5, 25])
# 12 days is our cutoff for logistic regression

In [ ]:
# Summary stats for elapsed time
elapsed_time_df.describe()

In [ ]:
# NEED TO REMOVE TIME OUTLIERS

In [ ]:
#Setting up a function for converting strings to number objects

loans_df.BORROWER_GENDERS.astype(str)

test_string = "female, female, female, female, female, female"

def female(txt):
    lst = txt.split(", ")
    count = 0
    for x in lst:
        if x == "female":
            count +=1 
    return count

def male(txt):
    lst = txt.split(", ")
    count = 0
    for x in lst:
        if x == "male":
            count +=1 
    return count

female(test_string)

In [ ]:
# Make a working copy of the loans_df dataframe
working_loans_df = loans_df.set_index
working_loans_df

In [ ]:
# Look at the gender column in working_loans_df
working_loans_df = loans_df['BORROWER_GENDERS'].dropna()
working_loans_df

In [ ]:
# Convert the datatypes for gender to string
working_loans_df.astype(str)
#working_loans_df.set_index(0)

In [ ]:
#Apply the male and female gender functions to the dataset, yields a number value for each field. 
# The results are the next cell.
#new_df = working_loans_df.set_index
new_df = working_loans_df.apply(male)
new_df2 = working_loans_df.apply(female)
new_df2

In [ ]:
#The result of gender counting is that 1.245 million values are female, 
#408,585 values are male, and only 246,605 data points are groups.  

new_df2.value_counts()

#Recommendations to consider:
# Reassign all values 2 or greater to a group
# Drop all group values to simply the data.
# Decision point here! :)

In [ ]:
# Gender distrubtion histogram.  0 is male, 1 is female, long tail are groups of varying composition.
new_df2.hist(range=[-5, 50])

In [ ]:
# Original df counts to compare to function results to make sure transformation did not alter data
loans_df['BORROWER_GENDERS'].value_counts()

In [ ]:
# COMBINE ALL VALUES GREATER THAN 2 TO GROUP

In [ ]:
# Add gender dataframe converted to numbers back into the database.
loans_new_df = loans_df.update(new_df2)

In [ ]:
#Gender values are now numeric
loans_df.head()

In [ ]:
# Look at elapsed time dataframe
elapsed_time_df

In [ ]:
#Add a new column with days from request to funding completion
#loans_new_df = loans_df.append(elapsed_time_df, ignore_index = True)
#loans_new_df
loans_df['FUNDING_TIME'] = elapsed_time_df

In [ ]:
#Confirm that Time to funding was added as a column to loans_df
loans_df.columns

In [ ]:
#dropping all unneeded columns 
#DOES THIS NEEDS TO BE UPDATED SO WE DON'T REMOVE TOO MUCH?  ???SHOUD WE BE REMOVING CURRENCY, REPAYMENT_INTERVAL???
loans_df.drop(['LOAN_ID', 'LOAN_NAME', 'DESCRIPTION','DESCRIPTION_TRANSLATED','IMAGE_ID', 'VIDEO_ID', 'LOAN_USE','COUNTRY_CODE', 'TOWN_NAME', 'CURRENCY_POLICY',
       'CURRENCY_EXCHANGE_COVERAGE_RATE', 'CURRENCY', 'PARTNER_ID', 'POSTED_TIME', 'PLANNED_EXPIRATION_TIME', 'DISBURSE_TIME','RAISED_TIME', 'LENDER_TERM', 'NUM_JOURNAL_ENTRIES', 'NUM_BULK_ENTRIES', 'BORROWER_NAMES','BORROWER_PICTURED', 'REPAYMENT_INTERVAL','DISTRIBUTION_MODEL'], axis=1, inplace=True)
loans_df.head()

In [ ]:
#saving tags column as a df  FOR FUTURE USE IN NATURAL LANUGUAGE PROCESSING ANALYSIS
tags_df = pd.DataFrame(loans_df["TAGS"])
tags_df.head(40)

In [ ]:
#pick a sample of the data as using all did not work for get dummies (10% used) ??? LOOKS LIKE 1%???
model_df = loans_df.sample(frac =.01)

# Prepare data for machine learning models

For logistic regression, our binary classification is that a successful borrowing event results in full funding within 12 days.  An unsucessful event would be funding taking longer than 12 days, as a reflection of less lender enthusiasm to fund the loan.  This removes the issue in the data that 99%+ of loans get funded and thus the data is very unbalanced if you just look at funding vs didn't fund.  

In [ ]:
# Create our features  NOT SURE ABOUT THIS VS THE TRAINING AND TESTING A FEW CELLS DOWN

X = model_df.copy()
X = X.drop('STATUS', axis=1)

# Create our target
y = model_df[['STATUS']]

In [ ]:
# Describe X

X.describe()

In [ ]:
# List out y

y

In [ ]:
# Check the balance of our target values  
# Used a calculated value of TIME TO FULL FUNDING using date stamps in prep for LOGISTIC REGRESSION

y['STATUS'].value_counts()

In [ ]:
# Encode Labels (DO WE USE THIS OR ONE HOT ENCODER?)

X = pd.get_dummies(X)
X.head()

In [ ]:
# ADD TRAIN-TEST SPLIT AFTER GETTING DUMMIES AND BEFORE SCALING, SO RIGHT HERE

In [ ]:
# SCALING STEP HERE
loans_scaled = StandardScaler().fit_transform(X)
print(loans_scaled[0:5])

In [ ]:
# PCA model intialization (can compare with 500 components)
pca = PCA(n_components=1000)

In [ ]:
# PCA fit
loans_pca = pca.fit_transform(loans_scaled)

In [ ]:
# transform PCA data to a DataFrame 
pca_df = pd.DataFrame(data=loans_pca, columns=["PC 1", "PC 2", "PC 3", "PC 4", "PC 5"])
pca_df.head()

In [ ]:
# Create k means model 
# *CAN USE OTHER MODELS ASIDE FROM K MEANS, AFTER PCA.  
# If English was a determinant, like language, then you need to have all languages to make the info complete.
# Or other category is "other languages", or top 3.  Or use Random Forest which takes in weak relationships and makes them stronger.
# Best to try every model we learned in the module.

inertia = []
k = list(range(1, 11))
# Calculate the inertia for the range of K values
for i in k:
   km = KMeans(n_clusters=i, random_state=0)
   km.fit(pcs_df)
   inertia.append(km.inertia_)

In [ ]:
# Create elbow curve using hvPlot
elbow_data = {"k" : k, "inertia":inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

In [ ]:
# Initialize the K-means model
model = KMeans(n_clusters=3, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

# Add the predicted class columns
pcs_df["class"] = model.labels_
pcs_df

In [ ]:
#create clustered df 
clustered_df = pd.merge(model_df, pcs_df, left_index=True, right_index=True)
clustered_df.head()

In [ ]:
# NEED SCREENSHOT OF THE PLOT FOR LATER BECAUSE THE GRADERS DON'T HAVE THE DATASET
# Plot the 3D-scatter
fig = px.scatter_3d(
clustered_df,
x="LOAN_AMOUNT",
y="FUNDING_TIME",
z="BORROWER_GENDERS",
color="class",
symbol="class",
hover_name="STATUS",
hover_data=["COUNTRY_NAME"],
width=800,)

fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [ ]:
# NEED TO RUN CONFUSION MATRIX ON PREDICTED CLUSTERS *AND* ORIGINAL Y VALUES, NOT THE K-MEANS Y VALUES.


In [ ]:
# Create hvplot scatter plot to view relationship
clustered_df.hvplot.scatter(
    x="COUNTRY_NAME",
    y="FUNDING_TIME",
    by="class")

# Run Logistic Regression on FUNDING_TIME and Analyze Results

# Run SVMs using PCA 1,000 components and Epsilon SVR and Analyze Results

# Run NLP on Tags DF